# Figures and analyses for propgage paper

Figures and statistical tests appear in the order that they do in the manuscript. Supplementary figures are at the end.

Note that we import a package from R for Hartigan's dip test, i.e. you will need a working installation of R configured to work with python to run that part of the notebook.

### Import packages

In [11]:
import os
import sys

import pandas as pd
import numpy as np

import math
import re
import string

from PhiSpyAnalysis import theils_u, DateConverter, printmd
from PhiSpyAnalysis import read_phages, read_gtdb, read_checkv, read_base_pp, read_categories, read_metadata, read_transposons
from Bio import Phylo


from scipy.stats import pearsonr, f_oneway, mannwhitneyu
import scipy as scp
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, LeaveOneGroupOut
from sklearn import metrics
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.multicomp import pairwise_tukeyhsd, tukeyhsd, MultiComparison
from statsmodels.multivariate.manova import MANOVA
from sklearn import decomposition
from sklearn.ensemble import RandomForestClassifier

# import rpy2.robjects as robjects
# from rpy2.robjects.packages import importr
# from rpy2.robjects import pandas2ri
# from rpy2.robjects import Formula

# for parsing collection dates
from dateutil.parser import parse, ParserError
import pytz

import subprocess
import gzip
import pickle as pkl
# import zstd

# import function dc which converts a date to a year and decimal (e.g. 1/1/1900 is 1900.00)
dc = DateConverter()


### Import plot tools and figure settings

In [3]:
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib as mpl
import matplotlib.gridspec as gridspec
import matplotlib.ticker as mtick
# from matplotlib.ticker import FuncFormatter
# from matplotlib.ticker import ScalarFormatter
formatter = mtick.ScalarFormatter(useMathText=True)
formatter.set_scientific(True)
from matplotlib.lines import Line2D
from matplotlib.colors import LogNorm, Normalize
import matplotlib.cm as cm
from matplotlib.colors import Normalize
from matplotlib import colors


colourDict = dict(zip(['rose', 'indigo', 'sand', 'green', 'cyan', 'wine', 'teal', 'olive', 'purple', 'pale_grey', 'black'],
                 ['#CC6677', '#332288', '#DDCC77', '#117733', '#88CCEE',
                    '#882255', '#44AA99', '#999933', '#AA4499', '#DDDDDD',
                    '#000000']))


fig_width_pt = 410.0  # Get this from LaTeX using \showthe\columnwidth
inches_per_pt = 1.0/72.27               # Convert pt to inch
golden_mean = (np.sqrt(5)-1.0)/2.0         # Aesthetic ratio
fig_width = fig_width_pt*inches_per_pt  # width in inches
fig_height = fig_width*golden_mean       # height in inches
fig_size =  [fig_width,fig_height]
params = {'backend': 'ps','axes.labelsize': 8,'font.size': 8,
          'legend.fontsize': 8,'xtick.labelsize': 8,'ytick.labelsize': 8,
          'text.usetex': True,'figure.figsize': fig_size, 'axes.linewidth':0.3}
mpl.rcParams['text.latex.preamble'] = [r'\usepackage[cm]{sfmath}',r'\usepackage{underscore}']
mpl.rcParams['font.family'] = 'sans-serif'
mpl.rcParams['font.sans-serif'] = 'cm'
mpl.rcParams['text.usetex']=False
mpl.rcParams['figure.dpi'] = 300

mpl.rcParams.update(params)

/var/folders/1l/n1k5w73d0gn1r0006j7fvt340000gn/T/ipykernel_31028/2838924867.py:32: MatplotlibDeprecationWarning: Support for setting an rcParam that expects a str value to a non-str value is deprecated since 3.5 and support will be removed two minor releases later.
  mpl.rcParams['text.latex.preamble'] = [r'\usepackage[cm]{sfmath}',r'\usepackage{underscore}']


### Some useful functions

In [4]:
class SeabornFig2Grid():

    def __init__(self, seaborngrid, fig,  subplot_spec):
        self.fig = fig
        self.sg = seaborngrid
        self.subplot = subplot_spec
        if isinstance(self.sg, sns.axisgrid.FacetGrid) or \
            isinstance(self.sg, sns.axisgrid.PairGrid):
            self._movegrid()
        elif isinstance(self.sg, sns.axisgrid.JointGrid):
            self._movejointgrid()
        self._finalize()

    def _movegrid(self):
        """ Move PairGrid or Facetgrid """
        self._resize()
        n = self.sg.axes.shape[0]
        m = self.sg.axes.shape[1]
        self.subgrid = gridspec.GridSpecFromSubplotSpec(n,m, subplot_spec=self.subplot)
        for i in range(n):
            for j in range(m):
                self._moveaxes(self.sg.axes[i,j], self.subgrid[i,j])

    def _movejointgrid(self):
        """ Move Jointgrid """
        h= self.sg.ax_joint.get_position().height
        h2= self.sg.ax_marg_x.get_position().height
        r = int(np.round(h/h2))
        self._resize()
        self.subgrid = gridspec.GridSpecFromSubplotSpec(r+1,r+1, subplot_spec=self.subplot)

        self._moveaxes(self.sg.ax_joint, self.subgrid[1:, :-1])
        self._moveaxes(self.sg.ax_marg_x, self.subgrid[0, :-1])
        self._moveaxes(self.sg.ax_marg_y, self.subgrid[1:, -1])

    def _moveaxes(self, ax, gs):
        #https://stackoverflow.com/a/46906599/4124317
        ax.remove()
        ax.figure=self.fig
        self.fig.axes.append(ax)
        self.fig.add_axes(ax)
        ax._subplotspec = gs
        ax.set_position(gs.get_position(self.fig))
        ax.set_subplotspec(gs)

    def _finalize(self):
        plt.close(self.sg.fig)
        self.fig.canvas.mpl_connect("resize_event", self._resize)
        self.fig.canvas.draw()

    def _resize(self, evt=None):
        self.sg.fig.set_size_inches(self.fig.get_size_inches())
        
def saveZstd(object, filename):
    bytesobject = pkl.dumps(object)
    file = zstd.compress(bytesobject,20)
    pkl.dump(file,open(filename,'wb'))
    
    
def openZstd(filename):
    fileObj = pkl.load(open(filename,'rb'))
    decompObj = zstd.decompress(fileObj)
    unpickleObj = pkl.loads(decompObj)
    
    return unpickleObj

class MathTextSciFormatter(mtick.Formatter):
    def __init__(self, fmt="%1.0e"):
        self.fmt = fmt
    def __call__(self, x, pos=None):
        s = self.fmt % x
        decimal_point = '.'
        positive_sign = '+'
        tup = s.split('e')
        significand = tup[0].rstrip(decimal_point)
        sign = tup[1][0].replace(positive_sign, '')
        exponent = tup[1][1:].lstrip('0')
        if exponent:
            exponent = '10^{%s%s}' % (sign, exponent)
        if significand and exponent:
            s =  r'%s{\times}%s' % (significand, exponent)
        else:
            s =  r'%s%s' % (significand, exponent)
        return "${}$".format(s)
    
def metagenomeStatus(x):
    if 'metagenome' in x:
        return "Yes" 
    else: 
        return "No"
    
def updateDate(x,magDateDict):
#     c = .isnan()
    if np.isnan(x['isolation_date']):
        y = magDateDict.get(x['assembly_accession'])
        return y
    else:
        y = x['isolation_date']
        return y


In [5]:
# from PhiSpyAnalysis import dtype_definitions 
# dtype_definitions.gtdb_dtypes()

# Import the datasets. 

We have two data sets: `small` is just 99 genomes and 1,561 phages and should run quickly for development. 
Max contig specifies the maximimum number of contigs. Set to 100 to ensure acceptable quality genomes.



In [6]:
# use_small_data=False
# maxContig = 100

# phagesdf = read_phages(use_small_data=use_small_data, maxcontigs=maxContig)

### Read the metadata

The metadata is from PATRIC. The phage data is from us.

**NOTE:** Some of the PATRIC data refers to specific chromosomes/fragments in the GenBank file (e.g. plasmid, chromosome), but that may not equate to our predictions, because we have used the whole GenBank file. The PATRIC metadata is redundant for many fields, and so we just keep the first entry for each NCBI Assembly.

In [7]:
# metadf = read_metadata(use_small_data=use_small_data)
# metadf = metadf.drop_duplicates()

### Merge the dataframes

First, select some columns we want to keep from PATRIC, and then merge the data frames.

In [8]:
# acccol = 'assembly_accession'
# interesting_cols = [acccol, 'isolation_site','taxon_id', 'isolation_country', 'latitude', 'longitude', 'altitude', 'depth',
#                     'other_environmental', 'host_name', 'host_gender', 'host_age', 'host_health', 
#                     'body_sample_site', 'body_sample_subsite', 'other_clinical', 'gram_stain', 'cell_shape',
#                     'motility', 'sporulation', 'temperature_range', 'optimal_temperature', 'salinity',
#                     'oxygen_requirement', 'habitat', 'disease', 'isolation_date','refseq_accessions','pathovar']

# few_interesting_cols = [acccol,  'isolation_country',  'gram_stain',  
#                     'motility', 'sporulation', 'temperature_range', 'habitat', 'disease', 'isolation_date']

# tempdf = metadf[interesting_cols]

# phagemeta = pd.merge(tempdf, phagesdf, how='right', left_on=acccol, right_on=acccol)
# phagemeta = phagemeta.dropna(subset=['Kept'])
# phagemeta['normalise_by_prok_length'] = phagemeta['Genome length'].values - phagemeta['bp prophage'].values
# phagemeta['Kept_normed'] = phagemeta['bp prophage'].values / phagemeta['normalise_by_prok_length'].values

### Read the GTDB taxonomy data

Genbank and PATRIC have NCBI taxonomy. Read in the GTDB data from the GTDB site.

Note that the GTDB dump (read_gtdb()) does not contain all genomes from Genbank. We used GTDB-tk to get taxonomic assignments of all "missing" genomes in our dataset; these are imported in gtdbMissing and then merged.

In [9]:
s = re.compile('^.__')
# u = re.compile('_')
def split_taxonomy(x):
    p  = x.split(';')
    if len(p) != 7:
        sys.stderr.write("We have {len(p)} fields in {x}, but we expected 7 fields")
    # remove the prepended taxonomic rank name (e.g d__) and any remaining underscores before returning.
    # return [u.sub(' ', l) for l in [s.sub('', m) for m in p] ]
    return [s.sub('', m) for m in p]

##### NOTE!!!! Grabbing GTDB data
For whatever reason there is a type error I can't resolve on the gtdb import function

I am using the "currentspeciesids" for the rest of the notebook, which is from QC'd GTDB data

If you want to use all of the genomes, there are about another 150k in the "gtdbmissing" list commented out below although some are duplicated with the new GTDB release, will clean up shortly

however it appears many of these are of lower quality (maybe fragmented) so not good for finding 'nice' prophages which were dropped due to a lack of phage DNA


In [10]:
tc = ['domain', 'phylum', 'class', 'order', 'family', 'genus', 'species']
gtdb = pd.read_csv('../data/bac120_metadata_r207.tsv.gz', compression='gzip', sep='\t',
                  usecols=['ncbi_genbank_assembly_accession', 'gtdb_taxonomy'])

tc = ['domain', 'phylum', 'class', 'order', 'family', 'genus', 'species']
gtdb = pd.concat(
    [gtdb, pd.DataFrame.from_records(gtdb['gtdb_taxonomy'].apply(split_taxonomy), columns=tc)],
    axis=1)
gtdb.rename(columns={'ncbi_genbank_assembly_accession':'assembly_accession'},inplace=True)
taxameta = gtdb[['assembly_accession', 'domain', 'phylum', 'class', 'order', 'family', 'genus', 'species']]

# gtdbMissing = pd.read_csv('../data/missing_taxa_april22.txt.gz',sep='\t',compression='gzip',
#                           names=['assembly_accession', 'domain', 'phylum', 'class', 'order', 'family', 'genus', 'species'],
#                          index_col=None)

# taxameta = taxameta.append(gtdbMissing)

currentspeciesids = taxameta.copy()


# taxameta = pd.merge(phagemeta,taxameta, on='assembly_accession', how='inner')

# print(phagemeta.shape[0] - taxameta.shape[0],'genomes were in the PhiSpy dataset where we don\'t have taxa information')
# print('There are a total of', taxameta.shape[0], 'genomes in the study')

## Create a balanced taxa sample from the ~45k dereplicated GTDB genomes

In [ ]:
# # get the genomes on the 'leaves' of the GTDB taxa tree
# tree = Phylo.read("../data/bac120_r202.tree", "newick")
# termList = []
# for leaf in tree.get_terminals(): 
#     termList += [leaf.name]

# # Get the species names for those genomes, create a new dataframe with accession and taxa name
# balancedTax = pd.DataFrame()
# balancedTax['assembly_accession'] = termList
# balancedTax['assembly_accession'] = balancedTax['assembly_accession'].str.replace('RS_', '')
# balancedTax['assembly_accession'] = balancedTax['assembly_accession'].str.replace('GB_', '')

# gtdbDump = pd.read_csv('../data/bac120_taxonomy_r202.tsv', sep='\t',header=None,
#                          names=['assembly_accession','domain','phylum',
#                                 'class','order','family','genus','species'])
# gtdbDump['assembly_accession'] = gtdbDump['assembly_accession'].str.replace('RS_', '')
# gtdbDump['assembly_accession'] = gtdbDump['assembly_accession'].str.replace('GB_', '')

# speciesDict = dict(zip(gtdbDump['assembly_accession'],gtdbDump['species']))
# balancedTax['species'] = balancedTax['assembly_accession'].map(speciesDict)

# for col in balancedTax.columns:
#     gtdbDump[col] = gtdbDump[col].str.replace('.__','',regex=True)
# balancedTax = balancedTax[['assembly_accession']].merge(gtdbDump,on='assembly_accession',how='left')

There are ~45k unique species names in the GTDB tree. Let's check how many are in our <100 contig genbank data.

Note that rerunning this cell will create a new random subset of the data.

In [ ]:
# tmp = taxameta.sample(frac=1).reset_index(drop=True).drop_duplicates(subset='species',keep='first')
# tmp = tmp.merge(balancedTax,on='species',how='right')

# # Create a list of dereplicated genomes with the correct number of sampled genomes
# newickList = list(tmp['assembly_accession_x'])
# newickList = [item for item in newickList if not(pd.isnull(item)) == True]


# print('We have',len(newickList),'species in our <100 contig genbank data')

### Assembly summaries

Use the genbank assembly summary to get additional metadata e.g. MAG status, as well as isolation date (proxied as sample date)

In [10]:
assembly_summary = pd.read_csv('../data/assembly_summary.txt.gz', compression='gzip',sep='\t',skiprows=1)
assembly_summary.rename(columns={'# assembly_accession':'assembly_accession'},inplace=True)

/var/folders/1l/n1k5w73d0gn1r0006j7fvt340000gn/T/ipykernel_7623/2691450898.py:1: DtypeWarning: Columns (21) have mixed types. Specify dtype option on import or set low_memory=False.
  assembly_summary = pd.read_csv('../data/assembly_summary.txt.gz', compression='gzip',sep='\t',skiprows=1)


#### Update dates and MAG status in the Genbank summary file

In [11]:
assembly_summarySub = assembly_summary[['assembly_accession','seq_rel_date','excluded_from_refseq']]
assembly_summarySub.loc[:,'excluded_from_refseq'] = assembly_summarySub.loc[:,'excluded_from_refseq'].astype(str)
assembly_summarySub.loc[:,'MAG status'] = assembly_summarySub['excluded_from_refseq'].apply(lambda x: metagenomeStatus(x))
assembly_summarySub.loc[:,'isolation_date'] = assembly_summarySub.seq_rel_date.apply(dc.convert_date)
assembly_summaryMag = assembly_summarySub[assembly_summarySub['MAG status'] == "Yes"]

# # Create lists of genomes which are MAGs, and the dates they were "isolated" (sampled)
magAccs = list(assembly_summaryMag['assembly_accession'])
magDateDict = dict(zip(assembly_summarySub['assembly_accession'],assembly_summarySub['isolation_date']))

/var/folders/1l/n1k5w73d0gn1r0006j7fvt340000gn/T/ipykernel_7623/4175757948.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  assembly_summarySub.loc[:,'excluded_from_refseq'] = assembly_summarySub.loc[:,'excluded_from_refseq'].astype(str)
/var/folders/1l/n1k5w73d0gn1r0006j7fvt340000gn/T/ipykernel_7623/4175757948.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  assembly_summarySub.loc[:,'MAG status'] = assembly_summarySub['excluded_from_refseq'].apply(lambda x: metagenomeStatus(x))
/var/folders/1l/n1k5w

### Find good candidates

In [12]:
# Read the genome phage stats file
phagestatsdf = pd.read_csv('../data/phage_stats.20220620.tsv', sep='\t')
phagestatsdf.rename(columns={'GENOMEID':'assembly_accession'},inplace=True)
phagestatsdf.head()

,assembly_accession,Genome Name,Contigs > 5kb,Genome Contigs,Number of Coding Sequences,Too short,Not enough phage hits,Kept,Note
0,GCA_019958295.1,Buchnera aphidicola strain Tabriz.4 Contig1000...,1.0,1224,1075,0,0,0,Too few ORFs
1,GCA_900128725.1,Buchnera aphidicola strain BCifornacula genome...,2.0,3,0,0,0,0,None of the features have protein sequences
2,GCA_008244535.1,Dictyoglomus thermophilum strain PYS_80_B,14.0,23,1876,8,5,4,Complete
3,GCA_008015785.1,Methylophilus methylotrophus isolate Bin_42_2 ...,94.0,94,1148,13,22,0,Complete
4,GCA_001735525.1,"Shewanella colwelliana strain CSB03KR, whole g...",33.0,52,4040,23,4,3,Complete


In [13]:
# Read the phispy contig metadata file (stats per individual phage)
allphagedf = pd.read_csv('../data/phage_locations_20220620.tsv', sep='\t', header=None)
allphagedf.rename(columns={0:'assembly_accession',1:'phispy contig',2:'start',3:'stop',4:'length',5:'ngenes',6:'notes'},inplace=True)
allphagedf.head()

,assembly_accession,phispy contig,start,stop,length,ngenes,notes
0,GCA_008244535.1,VTFL01000006.1,337,34176,33840,27,Dropped. No genes were identified as phage genes
1,GCA_008244535.1,VTFL01000004.1,75050,95353,20304,25,Kept
2,GCA_008244535.1,VTFL01000001.1,205725,224319,18595,23,Dropped. No genes were identified as phage genes
3,GCA_008244535.1,VTFL01000001.1,71619,92803,21185,23,Kept
4,GCA_008244535.1,VTFL01000004.1,132141,156532,24392,22,Kept


#### Get the total length of phage ("Kept") DNA in the genome and substitute zero back in if all of them are dropped

In [14]:
# group the df to split the kept vs not phage dna
groupeddf = allphagedf.copy() # duplicate
groupeddf = allphagedf.groupby(['assembly_accession','notes']).sum()  # sum for total bp of phage DNA per category

In [15]:
# iterate over the genomes which have "Kept" phage 
allgenomes = list(allphagedf['assembly_accession'].unique())
allnotes = ['Kept']
iterables = [allgenomes, allnotes]

# set up a df to append a length of zero whenever there are no "kept" phage
allrowindex = pd.MultiIndex.from_product(iterables, names=["assembly_accession", "notes"])
allrowdf = pd.DataFrame(0, columns=['length'], index=allrowindex)

# get indices where "Kept" is missing
already_present_index = groupeddf.index.intersection(allrowdf.index)
missing_index = allrowdf.index.difference(groupeddf.index)

# Append total phage length for all genomes in dataset, with 0 whenever there are no kept phage
totalphagecontent = groupeddf.append(allrowdf.loc[missing_index, :]).reset_index()
totalphagecontent = totalphagecontent[totalphagecontent['notes'] == 'Kept'].reset_index(drop=True)
totalphagecontent = totalphagecontent[['assembly_accession','length']]
totalphagecontent.head()

/var/folders/1l/n1k5w73d0gn1r0006j7fvt340000gn/T/ipykernel_7623/461416556.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  totalphagecontent = groupeddf.append(allrowdf.loc[missing_index, :]).reset_index()


,assembly_accession,length
0,GCA_000003135.1,48918
1,GCA_000003645.1,40298
2,GCA_000003925.1,268087
3,GCA_000003955.1,166292
4,GCA_000005825.2,93419


In [16]:
# Merge the length info back to our summary metadata frame
phagestatsdf = phagestatsdf.merge(totalphagecontent,on='assembly_accession', how='inner')
phagestatsdf.shape

(945551, 10)

### Merge the new phage stats data with NCBI assembly dump

In [17]:
assembly_summary.columns
assembly_summary['excluded_from_refseq'] = assembly_summary['excluded_from_refseq'].astype(str)
assembly_summary.loc[:,'MAG status'] = assembly_summary['excluded_from_refseq'].apply(lambda x: metagenomeStatus(x))


rankedlineagedf = pd.read_csv('../data/rankedlineage.dmp', sep='\t', 
                              names=['taxid', 'strain','ncbispecies','genus','order',
                                    'family','class','phylum','superphylum','kingdom'], low_memory=False)
rankedlineagedf = rankedlineagedf[rankedlineagedf['kingdom'] == 'Bacteria']
assembly_summary = assembly_summary.merge(rankedlineagedf[['taxid','ncbispecies','strain']], on='taxid',how='left')

assembly_summary['ncbispecies'].fillna(assembly_summary['strain'], inplace=True)

assembly_summary.shape

(949933, 25)

In [18]:
### Merge on NCBI assembly summary
mergedf = phagestatsdf.merge(assembly_summary, on='assembly_accession')
mergedf = mergedf.merge(currentspeciesids,on='assembly_accession',how='left')
mergedf.head()

,assembly_accession,Genome Name,Contigs > 5kb,Genome Contigs,Number of Coding Sequences,Too short,Not enough phage hits,Kept,Note,length,...,MAG status,ncbispecies,strain,domain,phylum,class,order,family,genus,species
0,GCA_008244535.1,Dictyoglomus thermophilum strain PYS_80_B,14.0,23,1876,8,5,4,Complete,79800,...,No,Dictyoglomus thermophilum,Dictyoglomus thermophilum,Bacteria,Dictyoglomota,Dictyoglomia,Dictyoglomales,Dictyoglomaceae,Dictyoglomus,Dictyoglomus thermophilum
1,GCA_008015785.1,Methylophilus methylotrophus isolate Bin_42_2 ...,94.0,94,1148,13,22,0,Complete,0,...,Yes,Methylophilus methylotrophus,Methylophilus methylotrophus,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,GCA_001735525.1,"Shewanella colwelliana strain CSB03KR, whole g...",33.0,52,4040,23,4,3,Complete,83562,...,No,Shewanella colwelliana,Shewanella colwelliana,Bacteria,Proteobacteria,Gammaproteobacteria,Enterobacterales,Shewanellaceae,Shewanella,Shewanella colwelliana
3,GCA_003044255.1,Shewanella putrefaciens strain WS13 chromosome...,1.0,1,4407,16,8,1,Complete,63450,...,No,Shewanella putrefaciens,Shewanella putrefaciens,Bacteria,Proteobacteria,Gammaproteobacteria,Enterobacterales,Shewanellaceae,Shewanella,Shewanella putrefaciens
4,GCA_003315425.1,Shewanella putrefaciens strain 97 Ga0227277_10...,36.0,45,4323,22,3,5,Complete,76951,...,No,Shewanella putrefaciens,Shewanella putrefaciens,Bacteria,Proteobacteria,Gammaproteobacteria,Enterobacterales,Shewanellaceae,Shewanella,Shewanella sp002966515


## Find worthy candidates

choose specs, defaults listed below

I didn't filter on "Complete" as these filters made it redundant

1.	1 contig in the genome
2.	over 2000 coding genes in the genome
3.	Candidatus in the NCBI species name (did the job with minimal complexity, happy to try and be smarter)
4.	Take the rarest 25 phyla from the subsequent filtered list (the top few phyla have many hundreds of genomes)
5.	Prophage needs to be > 40kb 
6.	Potential phage “Dropped” due to no phage genes identified


In [19]:
# top100specieslist = list(mergedf['organism_name'].value_counts().head(200).index)
contigsInGenome = 1
cdsInGenome = 2000
stringToFilter = 'Candidatus' 
rarestPhyla = 25
minProphageLength = 40000


# could use a phylum list filter (phylum is in ... instead here of candidatus in NCBI species name)
candidatuslist = list(mergedf[mergedf['ncbispecies'].str.contains(stringToFilter, na=False)]['assembly_accession'])

# start filtering the dataset down
subdf = mergedf[~mergedf['organism_name'].isin(candidatuslist)]
subdf = subdf[(subdf['Genome Contigs'] <= contigsInGenome) & (subdf['Number of Coding Sequences'] > cdsInGenome)]
print(subdf.shape[0], 'candidatus genomes with', contigsInGenome, 'contigs in genome and',cdsInGenome,'cds')


10148 candidatus genomes with 1 contigs in genome and 2000 cds


In [28]:
# grab rare phyla
rarephyla = subdf['phylum'].value_counts()[-rarestPhyla:].index
subdf = subdf[subdf['phylum'].isin(rarephyla)]
print(subdf.shape[0], 'genomes from the rarest phyla')

# filter based on length
genomesofinterestList = list(subdf['assembly_accession'])
interrogateSubsetDf = allphagedf[(allphagedf['assembly_accession'].isin(genomesofinterestList)) & 
                                 (allphagedf['length'] > minProphageLength) & 
                                 (allphagedf['notes'] == 'Dropped. No genes were identified as phage genes')].reset_index(drop=True)

print(interrogateSubsetDf.shape[0], 'genomes with "dropped" phages over', minProphageLength, 'bp')

summaryFilteredGenomeTable = mergedf[mergedf['assembly_accession'].isin(genomesofinterestList)]


121 genomes from the rarest phyla
46 genomes with "dropped" phages over 40000 bp


#### Save to file

,assembly_accession,Genome Name,Contigs > 5kb,Genome Contigs,Number of Coding Sequences,Too short,Not enough phage hits,Kept,Note,length,...,MAG status,ncbispecies,strain,domain,phylum,class,order,family,genus,species
72261,GCA_004801255.1,Desulfovibrio desulfuricans strain IC1 chromos...,1.0,1,2680,10,6,3,Complete,67806,...,No,Desulfovibrio desulfuricans,Desulfovibrio desulfuricans,Bacteria,Desulfobacterota_I,Desulfovibrionia,Desulfovibrionales,Desulfovibrionaceae,Desulfovibrio,Desulfovibrio desulfuricans_C
72354,GCA_900116045.1,Desulfovibrio piger isolate FI11049 genome ass...,1.0,1,2535,10,2,4,Complete,176572,...,No,Desulfovibrio piger,Desulfovibrio piger,Bacteria,Desulfobacterota_I,Desulfovibrionia,Desulfovibrionales,Desulfovibrionaceae,Desulfovibrio,Desulfovibrio piger_A
72454,GCA_000691605.1,"Bdellovibrio bacteriovorus strain 109J, comple...",1.0,1,3589,7,1,0,Complete,0,...,No,Bdellovibrio bacteriovorus,Bdellovibrio bacteriovorus,Bacteria,Bdellovibrionota,Bdellovibrionia,Bdellovibrionales,Bdellovibrionaceae,Bdellovibrio,Bdellovibrio bacteriovorus
72458,GCA_002208115.1,Bdellovibrio bacteriovorus strain SSB218315 ch...,1.0,1,3579,11,0,0,Complete,0,...,No,Bdellovibrio bacteriovorus,Bdellovibrio bacteriovorus,Bacteria,Bdellovibrionota,Bdellovibrionia,Bdellovibrionales,Bdellovibrionaceae,Bdellovibrio,Bdellovibrio bacteriovorus_C
72459,GCA_002872415.1,Bacteriovorax stolpii strain DSM 12778 chromos...,1.0,1,3742,5,6,0,Complete,0,...,No,Bacteriovorax stolpii,Bacteriovorax stolpii,Bacteria,Bdellovibrionota,Bacteriovoracia,Bacteriovoracales,Bacteriovoracaceae,Bacteriovorax,Bacteriovorax stolpii
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
647687,GCA_013004105.1,"Gemmatimonas sp. TET16 chromosome, complete ge...",1.0,1,4390,29,8,5,Complete,131511,...,No,Gemmatimonas groenlandica,Gemmatimonas groenlandica,Bacteria,Gemmatimonadota,Gemmatimonadetes,Gemmatimonadales,Gemmatimonadaceae,Gemmatimonas,Gemmatimonas sp013004105
647896,GCA_015709495.1,Chthonomonadaceae bacterium isolate H1_ARM1 ch...,1.0,1,2631,23,1,1,Complete,24691,...,Yes,Chthonomonadaceae bacterium,Chthonomonadaceae bacterium,Bacteria,Armatimonadota,Fimbriimonadia,Fimbriimonadales,Fimbriimonadaceae,H1-ARM1,H1-ARM1 sp001567425
648315,GCA_013752535.1,"Bdellovibrio sp. KM01 chromosome, complete gen...",1.0,1,3769,32,4,2,Complete,46433,...,No,Bdellovibrio sp. KM01,Bdellovibrio sp. KM01,Bacteria,Bdellovibrionota,Bdellovibrionia,Bdellovibrionales,Bdellovibrionaceae,Bdellovibrio,Bdellovibrio sp013752535
648637,GCA_002563855.1,Thermoflexus hugenholtzii strain G233 Ga007014...,1.0,1,2656,13,4,2,Complete,71117,...,No,Chloroflexi bacterium G233,Chloroflexi bacterium G233,Bacteria,Chloroflexota,Dehalococcoidia,Tepidiformales,Tepidiformaceae,Tepidiforma,Tepidiforma sp002563855


In [124]:
# list(mergedf[mergedf['assembly_accession'].isin(genomesofinterestList)])

In [27]:
interrogateSubsetDf

,assembly_accession,phispy contig,start,stop,length,ngenes,notes
0,GCA_004801255.1,CP036295.1,2153716,2198232,44517,38,Dropped. No genes were identified as phage genes
1,GCA_002872415.1,CP025704.1,1158874,1234669,75796,96,Dropped. No genes were identified as phage genes
2,GCA_002872415.1,CP025704.1,3657103,3711357,54255,39,Dropped. No genes were identified as phage genes
3,GCA_001273775.1,CP011801.1,1130180,1184283,54104,57,Dropped. No genes were identified as phage genes
4,GCA_000196815.1,FP929003.1,3819445,3875538,56094,66,Dropped. No genes were identified as phage genes
5,GCA_002951815.1,CP019454.1,56295,110436,54142,62,Dropped. No genes were identified as phage genes
6,GCA_000020485.1,CP001098.1,1774084,1832770,58687,36,Dropped. No genes were identified as phage genes
7,GCA_000017805.1,CP000804.1,5171089,5214761,43673,32,Dropped. No genes were identified as phage genes
8,GCA_000017805.1,CP000804.1,1775718,1816184,40467,31,Dropped. No genes were identified as phage genes
9,GCA_000023225.1,CP001629.1,2849308,2934306,84999,80,Dropped. No genes were identified as phage genes
